In [1]:

import tensorflow as tf

from keras.models import load_model

import ctypes
from six.moves import cPickle
import numpy as np
import time
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
logging.debug("test")
from keras.models import load_model



Using TensorFlow backend.


In [2]:
def load_CIFAR10_dataset(file_name):
    with open(file_name, 'rb') as f:                
        d = cPickle.load(f)
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
        d = d_decoded
        data = d['data']
        labels = d['labels']
        raw_float_data = np.array(data, dtype=float) / 255.0
        return raw_float_data, labels
    

def mAP(y_true, y_pred):
    
    y_true = np.array(labels).astype(np.int64)
    y_true = tf.identity(y_true)

    y_pred = np.array(y_pred).astype(np.float32)
    y_pred = tf.identity(y_pred) # np to tensor
    _, m_ap = tf.metrics.average_precision_at_k(y_true, y_pred, 1)
    sess = tf.Session()
    sess.run(tf.local_variables_initializer())
    tf_map = sess.run(m_ap)
    return tf_map


In [3]:
model = load_model('/home/vtpc/Documents/Alvils/tensorrt/keras_to_tensorflow/model.h5')
cifar_file = "/home/vtpc/Documents/Alvils/tensorrt/cifar-10-batches-py/test_batch"
imgs, labels = load_CIFAR10_dataset(cifar_file)
processed_imgs = np.transpose(imgs.reshape([-1, 3, 32, 32]), (0, 2, 3, 1))
print(processed_imgs.shape)

(10000, 32, 32, 3)


In [4]:
t0 = time.time()
output = model.predict(processed_imgs, 450)
t1 = time.time()
total_t = t1-t0 
print("time:" + str(total_t))
print(output.shape)



time:2.33928012848
(10000, 10)


In [5]:
accuracy = mAP(labels, output)
print(accuracy)

0.852


print(output[1])

In [6]:
print(np.argmax(output[2500]))
print(output[2500, 6])


4
0.022772942
